In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import codecs, json
import unicodedata
# pip install Unidecode  <OR> conda install Unidecode
import unidecode

In [15]:
#importing all csv files
domarar = pd.read_csv('csv/blak-domarar.csv', sep=';', header=0)
einstaklingar = pd.read_csv('csv/blak-einstaklingar.csv', sep=';', header=0)
forsvarsmenn = pd.read_csv('csv/blak-forsvarsmenn.csv', sep=';', header=0)
lid = pd.read_csv('csv/blak-lid.csv', sep=';', header=0)
lidimoti = pd.read_csv('csv/blak-lidimoti.csv', sep=';', header=0)
lidsmenn = pd.read_csv('csv/blak-lidsmenn.csv', sep=';', header=0)
lidsstjorar = pd.read_csv('csv/blak-lidsstjorar.csv', sep=';', header=0)
thjalfarar = pd.read_csv('csv/blak-thjalfarar.csv', sep=';', header=0)
mot = pd.read_csv('csv/blak-mot.csv', sep=';', header=0)

# drop all SyndarLids with an ID (SyndarlidID)
# (the reason for not dropping using SyndarLid is because I don't trust that column to be inserted correctly with [0,1])
lid = lid[lid['SyndarlidID'].isna()]
# then dropping those two columns because we don't want virtual teams
lid = lid.drop(columns=['SyndarLid', 'SyndarlidID'])

In [18]:
# All duplicated birthdays
duplicated_einstaklingar = einstaklingar[einstaklingar.duplicated(subset=['Nafn', 'Fdagur', 'Kyn'], keep=False)]
duplicated_fdagur_kyn_einstaklingar = einstaklingar[einstaklingar.duplicated(subset=['Fdagur', 'Kyn'], keep=False)]
#print(duplicated_fdagur_kyn_einstaklingar)

In [30]:
duplicate_dict = defaultdict(dict)

for index, row in duplicated_fdagur_kyn_einstaklingar.iterrows():
    full_name = row['Nafn']
    first_name = full_name.split()[0]
    first_name_lowercase = first_name.lower()
    first_name_to_english = unidecode.unidecode(first_name_lowercase)
    Fdagur_date = row['Fdagur'].split()[0]
    
    if first_name_to_english in duplicate_dict.keys():
        if Fdagur_date in duplicate_dict[first_name_to_english].keys():
            duplicate_dict[first_name_to_english][Fdagur_date].append(row.values)
        else:
            duplicate_dict[first_name_to_english][Fdagur_date] = [row.values]
    else:
        #if Fdagur does not exist ->
        duplicate_dict[first_name_to_english][Fdagur_date] = [row.values]
        

In [31]:
#dict(duplicated_fdagur_kyn_einstaklingar)

# all duplicates
#duplicate_dict

# all duplicates for lúðvík
duplicate_dict["ludvik"]

{'1969-03-31': [array([641, 'Lúðvik Kristinsson', '1969-03-31 00:00:00.000', 'kk ',
         'Völsungur', nan, nan, nan, nan, nan, nan, nan,
         '2000-04-04 08:24:44.110', 0.0], dtype=object),
  array([866, 'Lúðvík Kristinsson', '1969-03-31 00:00:00.000', 'kk ', nan,
         nan, nan, nan, nan, nan, nan, nan, '2001-04-17 12:12:41.047', nan],
        dtype=object),
  array([2299, 'Lúðvík Kristinsson', '1969-03-31 00:00:00.000', 'kk ', nan,
         nan, nan, nan, nan, nan, nan, nan, '2007-04-26 11:38:05.860', nan],
        dtype=object)],
 '1996-09-02': [array([4160, 'Lúðvík', '1996-09-02 00:00:00.000', 'kk ', nan, nan, nan,
         nan, nan, nan, nan, nan, '2013-04-28 10:50:35.960', 0.0],
        dtype=object),
  array([4167, 'Lúðvík Már Matthíasson', '1996-09-02 00:00:00.000', 'kk ',
         'HK', nan, nan, nan, nan, nan, nan, nan, '2013-09-03 21:00:29.950',
         0.0], dtype=object)]}

In [35]:
#duplicate_dict["ludvik"]['1969-03-31'][0]
#duplicate_dict["ludvik"]['1969-03-31'][1]

array([866, 'Lúðvík Kristinsson', '1969-03-31 00:00:00.000', 'kk ', nan,
       nan, nan, nan, nan, nan, nan, nan, '2001-04-17 12:12:41.047', nan],
      dtype=object)

In [36]:
lidsmenn

,MotID,LidID,EinstID,Timastimpill
0,1,18,24,2000-04-01 10:42:30.767
1,1,18,104,2000-04-01 08:59:30.420
2,1,18,272,2000-04-01 10:40:44.913
3,1,18,452,2000-04-01 09:01:19.177
4,1,18,483,2000-04-30 10:04:24.983
...,...,...,...,...
25847,250,6012,3908,2014-08-13 10:52:36.627
25848,250,6012,3909,2014-08-13 10:52:36.627
25849,250,6012,3910,2014-08-13 10:52:36.627
25850,250,6012,3911,2014-08-13 10:52:36.627


In [59]:
duplicate_dict["ludvik"]['1969-03-31'][0][0]
#for index, row in lidsmenn.iterrows():
 #   print(row["EinstID"])
duplicated_fdagur_kyn_einstaklingar["EinstID"]

6       2930
8         60
13      1400
14      1348
17      1964
        ... 
4485    1413
4490    1840
4491    1841
4492    2854
4493    3842
Name: EinstID, Length: 1821, dtype: int64

In [62]:
dict_duplicate_compare_team_members = defaultdict(dict)

for index, row in lidsmenn.iterrows():
    ids = row["EinstID"]
    if ids in duplicated_fdagur_kyn_einstaklingar["EinstID"]:
        # now we only view ids that exist for duplicated people
        if ids in dict_duplicate_compare_team_members.keys():
            dict_duplicate_compare_team_members[ids].append(row.values)
        else:
            dict_duplicate_compare_team_members[ids] = [0]

272
381
243
573
616
624
149
637
143
42
257
304
392
395
448
621
133
366
598
467
586
648
169
188
230
515
526
51
288
551
131
481
93
156
163
239
547
172
261
389
564
642
478
640
14
18
401
606
22
100
442
488
67
396
136
165
246
626
106
513
609
64
135
171
284
585
28
388
540
615
213
449
455
34
505
6
229
393
525
159
435
479
43
497
641
289
294
512
130
373
461
516
582
638
66
200
211
295
305
311
402
535
264
285
639
222
260
397
412
150
562
63
109
436
195
370
561
140
199
387
477
57
555
53
132
344
509
151
286
491
599
622
65
191
459
476
377
623
184
269
385
399
107
245
527
576
118
190
579
139
501
173
360
422
568
636
219
380
141
321
95
274
443
129
186
355
649
307
23
330
460
152
263
308
375
418
446
134
334
398
595
176
198
335
610
170
575
17
70
88
374
390
548
256
367
510
511
167
177
290
322
546
578
69
153
46
187
220
96
166
262
544
545
605
99
168
268
287
560
581
361
480
329
353
299
309
320
336
8
13
92
423
550
625
158
310
351
468
567
73
126
259
265
272
243
573
616
624
143
847
257
392
395
448
528
621
133
849


243
1640
525
705
932
1488
1489
100
200
1043
1471
1624
150
139
397
560
581
1475
1601
600
968
1591
76
361
952
1436
1588
54
1179
1222
1594
1595
395
621
962
1437
1557
1559
398
1251
621
6
322
393
952
150
1314
107
245
576
579
775
1602
335
610
1150
1252
1608
1609
1475
1601
256
511
1020
1607
528
1004
1005
1007
64
165
171
387
692
1074
304
360
1290
1291
1292
1067
129
355
1309
1599
28
213
455
130
723
732
140
199
638
726
728
947
299
320
336
461
733
1516
1637
1638
1639
65
191
476
774
1627
6
322
952
961
1437
1588
1131
1251
1252
30
109
1053
151
491
622
767
1334
335
398
610
353
769
262
544
678
1565
1566
803
807
513
983
149
637
1235
42
360
1291
525
153
1106
818
823
824
1044
829
830
1382
1251
1252
511
1607
1618
1608
131
165
380
505
1441
129
355
1309
1442
272
1390
637
1236
1287
1556
169
230
344
526
693
163
239
547
1801
563
705
932
1488
1489
136
165
246
64
135
387
626
1505
1685
28
213
455
1156
366
505
1261
1358
130
723
732
311
1039
1043
1624
34
222
260
1723
30
1053
1824
140
199
726
728
109
436
555
53
509


2226
2227
579
1716
1718
2045
2167
1781
2030
2031
2160
2044
2047
2051
6
525
1206
2167
2169
2042
2043
2080
2137
1698
957
1252
1648
692
1074
2049
30
1053
1772
299
336
461
1817
2354
158
567
1460
1798
169
767
1010
1017
1575
2231
150
1723
139
388
1857
1858
2301
167
177
1121
1454
1589
1303
1512
460
988
392
525
621
1206
399
256
511
528
1004
1005
1007
334
1701
1702
1704
1707
1784
1785
2091
2115
2042
2043
2080
623
1337
1754
307
957
1252
1336
1648
1939
692
1074
2049
272
637
381
2448
2376
2377
163
239
1011
642
1497
2459
18
2244
2388
2389
136
165
246
135
387
1946
2004
1068
1506
455
1427
34
195
505
1101
43
140
199
728
732
2416
133
1378
150
260
1314
1723
132
509
2373
623
1337
1556
1952
399
245
329
775
1715
1718
2441
139
501
129
355
1309
1599
398
1131
1251
170
548
935
51
388
1857
1858
2301
1940
167
177
1121
1454
1589
153
2318
2472
2473
897
899
903
802
807
353
769
299
309
336
461
598
890
1162
2272
2476
2477
1386
2003
2060
1044
2224
829
830
2225
2226
2227
889
1267
1458
54
1369
1422
2138
8
2051
2192
576


3092
264
3110
3166
829
830
2226
2227
3069
1044
3070
563
1027
1724
1530
3179
2769
3155
648
1054
3180
3181
169
1817
2773
3094
3095
3098
3136
3112
3114
3123
3125
3127
1542
1546
1547
1887
3211
2943
2944
2949
2958
2959
2960
2977
2979
2982
2986
3031
2998
3000
3001
3002
3029
3008
3009
3011
3013
3018
3026
3034
3035
723
2416
130
140
199
728
732
2074
2146
947
1338
163
239
547
1817
2773
264
153
2047
2049
2422
2454
2754
2030
2167
2168
2688
1054
1781
2160
2690
1782
2686
2692
3232
2044
2453
2753
692
1501
1938
2051
2042
3098
1599
3101
3123
3127
3233
1782
2688
2690
2692
2042
2725
2740
3098
776
2204
3217
341
2356
2488
2864
2865
3214
1422
1437
1578
1663
2326
2328
2329
2030
2688
2690
153
1106
167
177
1121
1454
1589
335
610
1131
1005
1007
1210
3204
3296
3383
3384
8
259
2047
2049
2454
2754
322
1578
2111
3222
1838
2906
3441
1290
3364
2745
623
1556
2809
2046
2421
2700
2042
2043
579
1716
1718
957
1498
3335
3336
3340
3341
3311
3320
2050
2297
3375
3378
3382
3381
3387
2127
398
610
1252
353
1338
2751
2756
2111
18

2877
678
1338
2047
2049
2454
2753
1462
3198
4121
322
1205
2111
3081
3289
3400
43
497
641
1153
1179
1584
1595
3837
4021
3132
3133
243
1230
1312
2105
1053
2056
1772
3220
321
910
2093
3134
4158
258
1982
2204
513
2814
3724
1358
2167
2168
1054
2030
2160
1437
2745
3465
412
622
767
1334
73
1043
2743
3148
2439
3086
3090
1782
2692
4107
905
2750
2751
3846
188
774
3874
2055
2700
3559
259
2044
2051
2453
2754
3951
261
642
1497
1590
1838
54
395
2359
2740
372
1818
1936
1939
307
1498
1648
1754
2431
2730
200
311
402
2041
2792
3467
3470
3476
4041
4042
2791
2894
2080
2901
2363
2796
2915
3100
4109
329
1715
1716
1718
2018
4025
952
961
2448
2748
3991
4108
1384
3693
2847
3323
3664
1368
1985
2013
2014
2015
239
890
1786
288
551
933
1823
2897
2144
2145
3290
555
1352
563
1027
1520
586
1530
4115
3607
3611
3612
3614
3329
3331
3497
3500
4031
4032
2976
3237
3258
3123
3267
3556
3856
3277
4057
4058
4060
3336
3341
3731
3815
3816
3378
3382
3387
4019
3311
3632
3070
3352
1336
3125
2046
2102
2374
3127
3233
3388
3389
3963
2

In [21]:
reverted_back_to_dict = dict(duplicate_dict)
#reverted_back_to_dict

In [23]:
file_path = "json/einstaklingar_map.txt" ## your path variable
#duplicate_dict_json = json.dump(duplicate_dict, codecs.open(file_path, 'w', encoding='utf-8'), separators=(';', ':'), sort_keys=True, indent=4) ### this saves the array in .json format
#json_obj = json.dumps(duplicate_dict, indent = 4)
#dumped = json.dumps(duplicate_dict, cls=NumpyEncoder)
#dumped
#pd.DataFrame(reverted_back_to_dict).to_csv(file_path, encoding='utf-8-sig')
#duplicate_dict_json = json.dump(reverted_back_to_dict, codecs.open(file_path, 'w', encoding='utf-8-sig'))

#json = json.dumps(reverted_back_to_dict)
#f = open(file_path,"w")
#f.write(str(reverted_back_to_dict))
#f.close()

=====================================================================================
=

In [187]:
#FINAL STEP (run after everything is done):

#duplicated people put into it's own csv to be browsed later
pd.DataFrame(duplicated_einstaklingar).to_csv("csv/new/duplicated-einstaklingar.csv", encoding='utf-8-sig')
pd.DataFrame(duplicate_dict).to_csv("json/duplicate-map.json", encoding='utf-8-sig')


#save as new csv inside csv/new
pd.DataFrame(domarar).to_csv("csv/new/blak-domarar.csv", encoding='utf-8-sig')
pd.DataFrame(einstaklingar).to_csv("csv/new/blak-einstaklingar.csv", encoding='utf-8-sig')
pd.DataFrame(forsvarsmenn).to_csv("csv/new/blak-forsvarsmenn.csv.csv", encoding='utf-8-sig')
pd.DataFrame(lid).to_csv("csv/new/blak-lid.csv", encoding='utf-8-sig')
pd.DataFrame(lidimoti).to_csv("csv/new/blak-lidimoti.csv", encoding='utf-8-sig')
pd.DataFrame(lidsmenn).to_csv("csv/new/blak-lidsmenn.csv", encoding='utf-8-sig')
pd.DataFrame(lidsstjorar).to_csv("csv/new/blak-lidsstjorar.csv", encoding='utf-8-sig')
pd.DataFrame(mot).to_csv("csv/new/blak-mot.csv", encoding='utf-8-sig')
pd.DataFrame(thjalfarar).to_csv("csv/new/blak-thjalfarar.csv", encoding='utf-8-sig')